### Thyroid_Disease_Predicition

Problem Statement:

Thyroid disease is a common cause of medical diagnosis and prediction, with an onset
that is difficult to forecast in medical research. The thyroid gland is one of our body's
most vital organs. Thyroid hormone releases are responsible for metabolic regulation.
Hyperthyroidism and hypothyroidism are one of the two common diseases of the thyroid
that releases thyroid hormones in regulating the rate of body's metabolism.

The main goal is to predict the estimated risk on a patient's chance of obtaining thyroid
disease or not.

In [1]:
import pandas as pd
import numpy

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
#import dtale
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler,LabelEncoder

import sklearn.impute

from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings('ignore')

In [3]:
data=pd.read_csv('hypothyroid.csv')
data.columns

Index(['age', 'sex', 'on_thyroxine', 'query_on_thyroxine',
       'on_antithyroid_medication', 'sick', 'pregnant', 'thyroid_surgery',
       'I131_treatment', 'query_hypothyroid', 'query_hyperthyroid', 'lithium',
       'goitre', 'tumor', 'hypopituitary', 'psych', 'TSH_measured', 'TSH',
       'T3_measured', 'T3', 'TT4_measured', 'TT4', 'T4U_measured', 'T4U',
       'FTI_measured', 'FTI', 'TBG_measured', 'TBG', 'referral_source',
       'Class'],
      dtype='object')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3772 entries, 0 to 3771
Data columns (total 30 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   age                        3772 non-null   object
 1   sex                        3772 non-null   object
 2   on_thyroxine               3772 non-null   object
 3   query_on_thyroxine         3772 non-null   object
 4   on_antithyroid_medication  3772 non-null   object
 5   sick                       3772 non-null   object
 6   pregnant                   3772 non-null   object
 7   thyroid_surgery            3772 non-null   object
 8   I131_treatment             3772 non-null   object
 9   query_hypothyroid          3772 non-null   object
 10  query_hyperthyroid         3772 non-null   object
 11  lithium                    3772 non-null   object
 12  goitre                     3772 non-null   object
 13  tumor                      3772 non-null   object
 14  hypopitu

In [5]:
data['hypopituitary'].value_counts()

f    3771
t       1
Name: hypopituitary, dtype: int64

All feature data types are object

In [6]:
data.describe()

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,Class
count,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,...,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772
unique,94,3,2,2,2,2,2,2,2,2,...,2,242,2,147,2,235,1,1,5,4
top,59,F,f,f,f,f,f,f,f,f,...,t,?,t,?,t,?,f,?,other,negative
freq,95,2480,3308,3722,3729,3625,3719,3719,3713,3538,...,3541,231,3385,387,3387,385,3772,3772,2201,3481


# Data Preprocessing
Seems some of the columns has '?' as empty values

In [7]:
# dtale.show(data)

Have to replace '?' to 'nan' to remove empty values,because dtypes of features are objects

In [8]:
for column in data.columns:
    count = data[column][data[column]=='?'].count()
    if count != 0:
        print(column, count)

age 1
sex 150
TSH 369
T3 769
TT4 231
T4U 387
FTI 385
TBG 3772


"TBG" has 37772 as null value it is good to drop and some columns too and unwanted columns 'TSH_measured','T3_measured','TT4_measured','T4U_measured','FTI_measured','TBG_measured'

In [9]:
data= data.drop(['TBG','TSH_measured','T3_measured','TT4_measured','T4U_measured','FTI_measured','TBG_measured'],axis =1)

In [10]:
for column in data.columns:
    count = data[column][data[column]=='?'].count()
    if count!=0:
        data[column] = data[column].replace('?',np.nan)
for column in data.columns:
    count = data[column][data[column]=='?'].count()
    if count==0:
        print(column, count)

age 0
sex 0
on_thyroxine 0
query_on_thyroxine 0
on_antithyroid_medication 0
sick 0
pregnant 0
thyroid_surgery 0
I131_treatment 0
query_hypothyroid 0
query_hyperthyroid 0
lithium 0
goitre 0
tumor 0
hypopituitary 0
psych 0
TSH 0
T3 0
TT4 0
T4U 0
FTI 0
referral_source 0
Class 0


'?' is replaced with 'Nan' Sucessfully

In [11]:
data.isna().sum()

age                            1
sex                          150
on_thyroxine                   0
query_on_thyroxine             0
on_antithyroid_medication      0
sick                           0
pregnant                       0
thyroid_surgery                0
I131_treatment                 0
query_hypothyroid              0
query_hyperthyroid             0
lithium                        0
goitre                         0
tumor                          0
hypopituitary                  0
psych                          0
TSH                          369
T3                           769
TT4                          231
T4U                          387
FTI                          385
referral_source                0
Class                          0
dtype: int64

## Replacing Categorical to numbers


In [12]:
data['sex'] = data['sex'].replace({'F' : 0, 'M' : 1})

for column in data.columns:
    if  len(data[column].unique())==2:
        data[column] = data[column].replace({'f' : 0, 't' : 1})
    elif len(data[column].unique())==1:
        data[column] = data[column].replace({'f' : 0})

data['referral_source'] = data['referral_source'].replace({'SVHC' : 1, 'SVI' : 2, 'STMW' : 3 , 'SVHD' :4 , 'other' :5})


data['Class'] = data['Class'].replace({'negative' : 0, 'compensated_hypothyroid' : 1,'primary_hypothyroid' :2,'secondary_hypothyroid':3})

data["Class"] = data["Class"].apply(lambda value : 1 if value >= 1 else 0)


imputer=sklearn.impute.KNNImputer(n_neighbors=3, weights='uniform',missing_values=np.nan)
new_array=imputer.fit_transform(data)
data=pd.DataFrame(data=np.round(new_array), columns=data.columns)

#### As 'Class' has some imbalance.... we can add all Hypothyroid as (1) one and negative as (0) seperate  output

In [13]:
data.tail()

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,tumor,hypopituitary,psych,TSH,T3,TT4,T4U,FTI,referral_source,Class
3767,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,2.0,3.0,131.0,1.0,117.0,5.0,0.0
3768,68.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,2.0,124.0,1.0,114.0,2.0,0.0
3769,74.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,2.0,112.0,1.0,105.0,5.0,0.0
3770,72.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,2.0,82.0,1.0,87.0,2.0,0.0
3771,64.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,2.0,99.0,1.0,92.0,5.0,0.0


In [14]:
imputer=sklearn.impute.KNNImputer(n_neighbors=3, weights='uniform',missing_values=np.nan)
new_array=imputer.fit_transform(data)
data=pd.DataFrame(data=np.round(new_array), columns=data.columns)

In [15]:
data.isnull().sum()

age                          0
sex                          0
on_thyroxine                 0
query_on_thyroxine           0
on_antithyroid_medication    0
sick                         0
pregnant                     0
thyroid_surgery              0
I131_treatment               0
query_hypothyroid            0
query_hyperthyroid           0
lithium                      0
goitre                       0
tumor                        0
hypopituitary                0
psych                        0
TSH                          0
T3                           0
TT4                          0
T4U                          0
FTI                          0
referral_source              0
Class                        0
dtype: int64

In [16]:
data["hypopituitary"].value_counts()

0.0    3771
1.0       1
Name: hypopituitary, dtype: int64

No null value

# Feature Selection

Seems some of the columns are not impacts on predicting output.As it the update data verson,previous versio n of the data has clear lables and records.As model concern feature selection has to be done......

In [17]:
y=data["Class"].copy()
X=data.drop('Class',axis=1).copy()
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.8,shuffle=True,random_state=123)

In [18]:
X_train.nunique()

age                           91
sex                            2
on_thyroxine                   2
query_on_thyroxine             2
on_antithyroid_medication      2
sick                           2
pregnant                       2
thyroid_surgery                2
I131_treatment                 2
query_hypothyroid              2
query_hyperthyroid             2
lithium                        2
goitre                         2
tumor                          2
hypopituitary                  2
psych                          2
TSH                           86
T3                            10
TT4                          229
T4U                            3
FTI                          220
referral_source                5
dtype: int64

In [19]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(10,'Score'))  #print 10 best features

                Specs         Score
16                TSH  77121.541469
20                FTI   3757.920430
18                TT4   3569.940747
17                 T3     37.389572
9   query_hypothyroid     26.338754
2        on_thyroxine     21.736074
1                 sex      7.381573
6            pregnant      4.430623
15              psych      2.929734
12             goitre      2.842287


We are taking 1st 10 feature that make impact on output

# New Data by Feature Selection

In [20]:
new_data = data.filter(['TSH','FTI','TT4','T3','query_hypothyroid','on_thyroxine','sex','pregnant','psych','Class'])

In [21]:
new_data.to_csv('new_Thyroid_data.csv', index=False)

In [22]:
new_data['pregnant'].value_counts()

0.0    3719
1.0      53
Name: pregnant, dtype: int64

In [23]:
#dtale.show(new_data)

In [24]:
y=new_data["Class"].copy()
X=new_data.drop('Class',axis=1).copy()
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.8,shuffle=True,random_state=123)

In [25]:
y.value_counts()

0.0    3481
1.0     291
Name: Class, dtype: int64

### Seems imbalanced class,RandomOverSample is done to over come this

In [28]:
from imblearn.over_sampling import RandomOverSampler

os=RandomOverSampler(sampling_strategy='all')

X_train_res,y_train_res=os.fit_resample(X_train,y_train)

In [38]:
from collections import Counter
Counter(y_train_res)

Counter({0.0: 2791, 1.0: 2791})

In [ ]:
import joblib as jb

In [ ]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV (rfc, forest_params, cv = 10, scoring='accuracy')
clf.fit(X_train, y_train)

In [ ]:
print(clf.best_params_)

print(clf.best_score_)

In [ ]:
X_train.shape,y_train.shape

Nearly 98% of accuracy

In [ ]:
model=RandomForestClassifier( n_estimators=150,
                               criterion='gini',
                               max_depth=10)
model.fit(X_train,y_train)

from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
y_pred=model.predict(X_test)
cf_matrix=np.array(confusion_matrix(y_test,y_pred))
print(cf_matrix)
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print(format(classification_report(y_test,y_pred)))

In [ ]:
y_pred=.predict([[12.0,87.0,99.0,2.0,1.0,0.0,0.0,0.0,0.0]])
y_pred

In [ ]:
import joblib as jb